# Composite AutoML

In [1]:
import pandas as pd

df = pd.read_csv('tmp/extracted_data_processed.csv')
X = df.iloc[:,:-1]
X = X.loc[:, ~X.columns.str.contains('^Unnamed')]
y = df.iloc[:,-1]

X.head()

,id,Zone16_Pressure__ratio_beyond_r_sigma__r_0.5,Zone26_Pressure__number_cwt_peaks__n_5,Zone4_Pressure__quantile__q_0.8,Zone7_Pressure__quantile__q_0.8,Zone31_Pressure__kurtosis,"Zone9_Pressure__fft_coefficient__coeff_31__attr_""imag""",Zone15_Pressure__kurtosis,Zone15_Pressure__quantile__q_0.7,Zone23_Pressure__skewness,...,"Zone14_Pressure__agg_linear_trend__f_agg_""mean""__chunk_len_50__attr_""stderr""","Zone10_Pressure__fft_coefficient__coeff_88__attr_""abs""","Zone31_Pressure__fft_coefficient__coeff_39__attr_""imag""",Zone24_Pressure__skewness,Zone17_Pressure__time_reversal_asymmetry_statistic__lag_1,"Zone31_Pressure__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.4","Zone2_Pressure__augmented_dickey_fuller__attr_""teststat""","Zone31_Pressure__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0",Zone26_Pressure__ratio_value_number_to_time_series_length,"Zone9_Pressure__fft_coefficient__coeff_30__attr_""imag"""
0,0,0.152859,26.0,90414.74,23210.18,1.178000,201211.715148,0.180995,18795.44,1.207939,...,1112.369466,76492.559938,255785.698246,2.473288,4.773759e+11,135393.084430,-1.086534,85.208147,0.325554,244955.679500
1,2,0.094170,23.0,93441.16,21939.86,2.641700,180946.077666,1.282848,16444.71,1.487467,...,566.148649,74394.719607,-98428.832501,3.445868,5.522429e+11,30438.276291,-2.317037,42.949780,0.382287,212275.353946
2,5,0.136808,33.0,43145.80,12085.70,1.567874,432280.949243,0.114999,11725.80,1.189575,...,604.656515,126717.303704,70494.515982,3.155242,6.161691e+11,78916.292213,0.628782,50.660506,0.378936,399243.625839
3,11,0.112735,25.0,41329.90,13174.00,3.252763,207945.724593,2.822502,10678.11,1.811675,...,322.081596,104570.553883,245829.624233,2.818438,6.086911e+11,27339.274344,0.894001,34.879843,0.410230,193371.775057
4,12,0.148026,24.0,38397.98,13821.24,-0.219816,274288.012114,0.325279,13051.98,1.084620,...,304.019976,82506.913319,238733.932588,2.321429,4.830278e+11,23691.742285,2.030197,50.837662,0.331140,256213.407809


In [2]:
# code
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

seed = 0
folds = 10
algorithm = RandomForestClassifier(n_estimators=100, random_state=seed)

def get_score(X, y):    
    # Cross Validation
    score = cross_val_score(algorithm, X, y, cv=folds).mean()
    print("Cross Validation Score: "+ str(score*100) + "%") 
    # Train-Test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)
    algorithm.fit(X_train, y_train)
    y_pred = algorithm.predict(X_test)
    print(classification_report(y_test, y_pred))

get_score(X, y)

Cross Validation Score: 76.5%
              precision    recall  f1-score   support

           0       0.78      0.82      0.80        34
           1       0.80      0.75      0.77        32

    accuracy                           0.79        66
   macro avg       0.79      0.79      0.79        66
weighted avg       0.79      0.79      0.79        66



### TPOT

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3)

# generations: int, optional (default=100)
#    Number of iterations to the run pipeline optimization process. Must be a positive number.
#    Generally, TPOT will work better when you give it more generations (and therefore time) to optimize the pipeline.
#    TPOT will evaluate population_size + generations × offspring_size pipelines in total. 
tpot = TPOTClassifier(generations=50, population_size=50, n_jobs=8, verbosity=2)
tpot.fit(X_train, y_train)

Generation 1 - Current best internal CV score: 0.8353767560664112


In [ ]:
tpot.score(X_test, y_test)

In [ ]:
tpot.export('tmp/tpot_composite_pipeline.py')